<a href="https://colab.research.google.com/github/desireedisco/MSDS-Machine-Learning-Supervised/blob/main/1_Data_Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
#Notebook for Data Preprocessing
---


This notebook is for data preprocessing. We will do the following:
* Load file, drop index, drop na, drop duplicates
* Separate out the email links, web links, hashtag, mentions.
* Clean the data leakage problem in the text column. There are also some minor problems in the title column that will be fixed.
* Count the number of sentences and calculate the mean sentence length.
* Drop the foreign language rows as determined by langdetect
* Check to see of the remaining tokens which are recognized as words and calculate the non-recognized word percentage
* Calculate the title and text similarity


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#install langdetect for when we separate out foreign language rows
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=3b36d697e6a8aef223cd5cfcb652a10ef80d2d5d73a22fbad4ed654a3496fa1f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [7]:
import pandas as pd
import numpy as np
import re
import nltk

from langdetect import detect
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from gensim.models import Word2Vec

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

***
##Load file, drop index, drop na, drop duplicates
***

In [55]:
# read csv file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning - Supervised Learning/project/ProjectData/WELFake_Dataset.csv')
data.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO],No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others o...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE “PEACEFUL” PROTESTERS…In Her Home State Of North Carolina [VIDEO],"Now, most of the demonstrators gathered last night were exercising their constitutional and protected right to peaceful protest in order to raise issues and create change. Loretta Lynch aka Er...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Christian conversion to woo evangelicals for potential 2016 bid",A dozen politically active pastors came here for a private dinner Friday night to hear a conversion story unique in the context of presidential politics: how Louisiana Gov. Bobby Jindal traveled f...,0
4,4,SATAN 2: Russia unvelis an image of its terrifying new ‘SUPERNUKE’ – Western world takes notice,"The RS-28 Sarmat missile, dubbed Satan 2, will replace the SS-18 Flies at 4.3 miles (7km) per sec and with a range of 6,213 miles (10,000km) The weapons are perceived as part of an increasingly ag...",1


In [56]:
# show dataframe info
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB
None


**Note on matching Real and Fake labels:**
In the dataset description the authors state that there are 72,134 news articles with 35,028 real and 37,106 fake news articles. The authors then go on to state the labels are labeled as follows: 0=fake and 1=real. The two statements are contradictory based on the following label counts. Upon further inspection of the data and following the authors first statement of '35,028 real and 37,106 fake news articles', I am following the mapping for the labels as 0=real and 1=fake.

In [57]:
# show the number of rows labeled 1 and number of rows labeled 0
print(data['label'].value_counts())

# show unique count for title and text
print(data.describe(include=['object']))

label
1    37106
0    35028
Name: count, dtype: int64
                                                       title   text
count                                                  71576  72095
unique                                                 62347  62718
top     Factbox: Trump fills top jobs for his administration       
freq                                                      14    738


In [58]:
# drop dataset index column
data = data.drop(columns=['Unnamed: 0'])

**Check to see how many rows are null and what the percentage of total data is before dropping rows.**

In [59]:
# show the number of rows that are null
for col in data.columns:
    print(f'{data[col].isnull().sum()} rows are null in ' + col)

# show what percentage of total is null for each column
for col in data.columns:
    print(col, f'{round(data[col].isnull().sum() / data.shape[0] * 100, 2)} % is null')

558 rows are null in title
39 rows are null in text
0 rows are null in label
title 0.77 % is null
text 0.05 % is null
label 0.0 % is null


In [60]:
#drop all null values
data = data.dropna().reset_index(drop=True)

**Split the data by label and check and drop duplicate 'text' columns. Then combine to check for duplicates**

I did this because I did not want to drop duplicates that had multiple labels attached. If there were any duplicate articles that had different labels then I wanted to know about that. I this dataset the only duplicate with multiple labels was a blank text row.

In [61]:
#split the data by label to look at unique count in case there are duplicates label both real and fake
real_data = data[data['label'] == 0]
fake_data = data[data['label'] == 1]
print(real_data.describe(include=['object']))
print(fake_data.describe(include=['object']))

                                                       title  \
count                                                  35028   
unique                                                 34409   
top     Factbox: Trump fills top jobs for his administration   
freq                                                      14   

                                                                                                                         text  
count                                                                                                                   35028  
unique                                                                                                                  34621  
top     Killing Obama administration rules, dismantling Obamacare and pushing through tax reform are on the early to-do list.  
freq                                                                                                                       58  
                                       

In [62]:
#drop duplicate text stories
real_data = real_data.drop_duplicates(subset=['text']).reset_index(drop=True)
fake_data = fake_data.drop_duplicates(subset=['text']).reset_index(drop=True)

# combine the separate labels to one to check for duplicates
data = pd.concat([real_data, fake_data], ignore_index=True)
print(data.describe(include=['object']))

                                                       title   text
count                                                  62201  62201
unique                                                 61400  62200
top     Factbox: Trump fills top jobs for his administration       
freq                                                      14      2


**As you can see the duplicate with multiple labels had blank text column.**

In [63]:
# display duplicate rows that have 2 different labels
print(data[data['text'].duplicated(keep=False)])

                                                                                 title  \
920                                                     Graphic: Supreme Court roundup   
34626  HOUSE INTEL CHAIR On Trump-Russia Fake Story: “No evidence of anything” [Video]   

      text  label  
920             0  
34626           1  


In [64]:
# they have blank text so drop rows
data = data.drop_duplicates(subset=['text'], keep=False).reset_index(drop=True)

# check total counts and unique counts
print(data.describe(include=['object']))

# display value counts for labels
data['label'].value_counts()

#value counts should be
# 0 - 34620
# 1 - 27579

                                                       title  \
count                                                  62199   
unique                                                 61398   
top     Factbox: Trump fills top jobs for his administration   
freq                                                      14   

                                                                                                                                                                                                           text  
count                                                                                                                                                                                                     62199  
unique                                                                                                                                                                                                    62199  
top     A dozen politically active pastors came h

,count
label,
0,34620
1,27579


**Shuffle and reindex dataset**

In [65]:
# shuffle dataframe and reset index
data = data.sample(frac=1).reset_index(drop=True)

***
##Clean Title - Source Data Leakage
***

In the data_leakage notebook we demonstrated that some source info that was embedded in the 'text' column influenced the model. Because not all articles have source info, it was unduly providing bias in the model. I looked through the 'title' column and decided there was similar data that could influence the model: therefore, I decided to remove that as well. I just want to focus on the actual title and actual text, not the sources of the articles. We could use the sources as a feature if all the articles have source information. In the current dataset, only the real news includes sources so it needs to be removed.

In [66]:
def clean_title_leakage(title):
  # making sure title is a string
  title = str(title)

  # get rid of New York Times and Breitbart reference
  pattern_leakage = r" - The New York Times$| - Breitbart$"
  # find all occurrences not just the first one
  match_lst = re.findall(pattern_leakage, title)
  # if match then substitue for ''
  if match_lst:
    title = re.sub(pattern_leakage, '', title)
    #print(match_lst)

  return title

In [67]:
# create a new column for the clean title and apply the clean_title_leakage method
data['title_clean'] = data['title'].apply(clean_title_leakage)

In [68]:
# just as well as the real news provides source information the fake news refers to a source of video so I decide to clean this as well
def clean_title_video(title):

  # get rid of Video reference - easy match pattern
  pattern_video = r"\[VIDEO\]|\(VIDEO\)"

  # find all matches and ignore case
  match_lst = re.findall(pattern_video, title, re.IGNORECASE)

  if match_lst:
    title = re.sub(pattern_video, '', title, flags=re.IGNORECASE)
    #print(match_lst)

  # search for more match patterns to get rid of all references to (Video)
  pattern_video_re = r"\[video[a-z 0-9/,+-].*\]|\[[a-z 0-9/,+-].*video\]|\(video[a-z 0-9/,+-].*\)|\([a-z 0-9/,+-].*video\)|\([a-z 0-9/,+-].*videos\)"
  match_lst_re = re.findall(pattern_video_re, title, re.IGNORECASE)
  if match_lst_re:
    title = re.sub(pattern_video_re, '', title, flags=re.IGNORECASE)
    #print(match_lst_re)

  return title

In [69]:
# cleaning the 'title_clean column of the (Video) references which is skewed to fake news
data['title_clean'] = data['title_clean'].apply(clean_title_video)

In [70]:
# increase column width to display wider columns
pd.set_option('display.max_colwidth', 200)

In [71]:
# check to see changes have been made
df_show = data[['title','title_clean']]
print(df_show.head)

<bound method NDFrame.head of                                                                                              title  \
0                                                                           Oh, What a Lovely War!   
1                               Deported Italian Mobster Caught Sneaking Across U.S.-Mexico Border   
2                                      China lodges protest after Trump call with Taiwan president   
3                                                The US May Soon Face an Apocalyptic Seismic Event   
4      JOY BEHAR Still Claims Clinton Won…BUT Wore Bizarre Mourning Item When Hillary Lost [Video]   
...                                                                                            ...   
62194                         Senate Gives Trump Jr. Ultimatum: Respond By Friday Or Face Subpoena   
62195                             Pence: I Don't Understand 'The Basis' of Michelle Obama's Claims   
62196                             South Africa's Zum

***
##Clean Text
***

In [72]:
# clean out newline reference
data['text_clean'] = data['text'].str.replace('\n', ' ')

**Extract email links**

In [73]:
# method to extract email links from text and put in another data frame column
def extract_email_links(text):
  emails = ''

  # get rid of web links - match for web links
  pattern_email = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
  match_lst = re.findall(pattern_email, text)
  # if match we will substitute email for '' in text_clean and add to emails list
  if match_lst:
    text = re.sub(pattern_email, '', text)
    #print(match_lst)
    emails = match_lst
  # return text_clean and emails list
  return pd.Series({'text_clean': text, 'email': emails})

In [74]:
# get text_clean striped of emails and put emails in separate column
data[['text_clean', 'email']] = data['text_clean'].apply(lambda x: pd.Series(extract_email_links(x)))

In [75]:
# show changes
data.loc[data['email'].str.len()>0,['text_clean', 'email']].head()

,text_clean,email
157,"Behind the headlines - conspiracies, cover-ups, ancient mysteries and more. Real news and perspectives that you won't find in the mainstream media. Browse: Home / U.S. Elections “November Chaos”: ...",[cyberpatriot@hotmail.com]
449,"21st Century Wire says This week s documentary film curated by our editorial team at 21WIRE. By August 1945, the Allied Manhattan Project had successfully detonated an atomic device in the New Mex...",[MEMBER@21WIRE.TV]
473,"It is 2007, and I am an undergraduate at the University of Tehran. I'm very particular. I take notes with Staedtler Triplus Fineliner pens, in purple and green, and on this particular day I've run...",[firstperson@vox.com]
687,"VINEYARD OF THE SAKER T he AngloZionst Empire’s propaganda machine, otherwise known as the corporate media, has had great difficulties deciding what it should say about the Russian naval task forc...",[editor@greanvillepost.com]
1599,"HERE S THE WIKILEAKS LINK TO ALL REFERENCES TO TOM PEREZ: TOM PEREZ WIKILEAKSHERE S ONE EMAIL TO JOHN PODESTA FROM PEREZ VIA WIKILEAKS:From: To: , Date: 2016-02-05 03:10 Subject: Observations fro...","[tomperez1@verizon.net, john.podesta@gmail.com, maya@mayalharris.com]"


**Extract web links**

In [76]:
# method to extract web links and put in another dataframe column
def extract_web_links(text):
  links = ''

  #get rid of web links - we try to match a number of different patterns to try to capture all the weblinks in the text column
  pattern_html = r"https?://(?:www\.)?[a-zA-Z0-9./?=&_-]+|pic.twitter.com[a-zA-Z0-9./]+|[a-zA-Z0-9./]+(?:\.com|\.org)/[a-zA-Z0-9/\-.]+|[a-zA-Z0-9./]+(?:\.com|\.org)"
  match_lst = re.findall(pattern_html, text)
  # if match we will substitute web link for '' in text_clean and add to web link list
  if match_lst:
    text = re.sub(pattern_html, '', text)
    #print(match_lst)
    links = match_lst

  # return text_clean and web link list
  return pd.Series({'text_clean': text, 'links': links})

In [77]:
# get text_clean striped of web links and put web links in separate column
data[['text_clean', 'links']] = data['text_clean'].apply(lambda x: pd.Series(extract_web_links(x)))
# record link count in separate column
data['link_count'] = data['links'].apply(lambda x: len(x))

In [78]:
# show changes
data.loc[data['links'].str.len()>0,['text_clean', 'links']].head()

,text_clean,links
7,"The Supreme Court sided with women on freedom and the right to choose today with their ruling on Texas restrictive abortion law. In a 5-3 ruling, they said that the arguments that the law merely ...","[pic.twitter.com/2KRpiFIMHx, https://t.co/sFC0tMHD6X, https://t.co/OIQJvJXOQK]"
21,"21st Century Wire says Amid great mainstream media and Democratic Party fanfare, Hillary Clinton s candidacy has been based on a claim she is most experienced candidate in history, and highligh...",[https://api.soundcloud.com/tracks/291631622]
30,"link Hey People: I always enjoy your threads, though I think you focus too much on a singular belief issue or pattern, and then it becomes self affirming for you in your research. First, this: But...",[Dictionary.com]
32,Did Hillary forget about her deplorable supporters who beat innocent Trump supporters bloody at one rally after another only a few months ago? But WE are the #BasketOfDeplorables? Share this. Be...,[pic.twitter.com/vRN2GqIYzo]
35,Election crossroads: Socialism or capitalism? Exclusive: Jane Chastain explains impact of Dems' 'living wage' proposal Published: 49 mins ago About | | Archive Jane Chastain is a Southern Californ...,[JaneChastain.com]


**Extract mentions**

In [79]:
# method to extract mentions and put in another dataframe column
def clean_mentions(text):
  mentions = ''

  # get rid of mentions in text and keep mentions in a separate column
  pattern_mentions = r"@\w+"
  match_lst = re.findall(pattern_mentions, text)
  # if match we will substitute mentions for '' in text_clean and add to mention list
  if match_lst:
    text = re.sub(pattern_mentions, '', text)
    #print(match_lst)
    mentions = match_lst

  # return text_clean and mention list
  return pd.Series({'text_clean': text, 'mentions': mentions})

In [80]:
# get text_clean striped of mentions and put mentions in separate column
data[['text_clean', 'mentions']] = data['text_clean'].apply(lambda x: pd.Series(clean_mentions(x)))
# new column with mention count
data['mentions_count'] = data['mentions'].apply(lambda x: len(x))
# turn mention list into string so we can vectorize latter on if we want
data['mentions'] = data['mentions'].apply(lambda mention_lst: ' '.join([str(mention) for mention in mention_lst]))

In [81]:
# show changes
data.loc[data['mentions'].str.len()>0,['text_clean', 'mentions']].head()

,text_clean,mentions
7,"The Supreme Court sided with women on freedom and the right to choose today with their ruling on Texas restrictive abortion law. In a 5-3 ruling, they said that the arguments that the law merely ...",@EagleGraphs @ConservTexas @petr3pan @Kristenn_2 @JacobBotzman @treetopangel @SteveDaines @Scotus @AndrewHolter
14,"The newspaper USA Today ran an article on Monday claiming that Donald Trump’s election last November has led to the hijab, commonly known as a symbol of oppression in the Muslim world, becoming a ...",@USATODAY @ben_kew @breitbart
21,"21st Century Wire says Amid great mainstream media and Democratic Party fanfare, Hillary Clinton s candidacy has been based on a claim she is most experienced candidate in history, and highligh...",@21WIRE
32,Did Hillary forget about her deplorable supporters who beat innocent Trump supporters bloody at one rally after another only a few months ago? But WE are the #BasketOfDeplorables? Share this. Be...,@TallahForTrump
38,"Lena Dunham claims (claims!) she is losing weight because Donald Trump has convinced her not to eat. The Catholic church won’t declare a person a saint until after their death, but if this extraor...",@Nero @breitbart


**Extract hashtags**

In [82]:
# method to extract hashtags and put in another dataframe column
def clean_hashtags(text):
  hashtags = ''
  hashtag_count = 0

  # get rid of hashtags in text and keep mentions in a separate column
  pattern_hashtags = r"#\w+"
  match_lst = re.findall(pattern_hashtags, text)

  # if match we will substitute mentions for '' in text_clean and add to mention list
  if match_lst:
    text = re.sub(pattern_hashtags, '', text)

    hashtags = match_lst
    hash_clean = ''

    # we want to search through hashtag list for #1,#2, or other strings with just numbers after # because there is a lot of them and they are not hashtags
    for index, hashtag in enumerate(hashtags):
      # if match is a number string do nothing otherwise add to hashtag list
      if re.search(r"#\d+$", hashtag):
        pass
      else:
        hash_clean = hash_clean + ' ' + hashtag
    # strip left space
    hashtags = hash_clean.lstrip()
    #print(hashtags)

    # count the number of hashtags in list
    hashtag_count = len(hashtags.split(' '))

  # return text_clean and hashtag list in string form and return hashtag count
  return pd.Series({'text_clean': text, 'hashtags': hashtags, 'hashtag_count': hashtag_count})

In [83]:
# process text to return 'text_clean', hashtag list, and hashtag count
data[['text_clean', 'hashtags', 'hashtag_count']] = data['text_clean'].apply(lambda x: pd.Series(clean_hashtags(x)))

In [84]:
# show changes
data.loc[data['hashtags'].str.len()>0,['text_clean', 'hashtags']].head()

,text_clean,hashtags
7,"The Supreme Court sided with women on freedom and the right to choose today with their ruling on Texas restrictive abortion law. In a 5-3 ruling, they said that the arguments that the law merely ...",#SCOTUS #Texas #Abortion #Trump #SCOTUS #aca #texas #abortion #SCOTUS
14,"The newspaper USA Today ran an article on Monday claiming that Donald Trump’s election last November has led to the hijab, commonly known as a symbol of oppression in the Muslim world, becoming a ...",#MuslimWomensDay
32,Did Hillary forget about her deplorable supporters who beat innocent Trump supporters bloody at one rally after another only a few months ago? But WE are the ? Share this. Because we aren't! B...,#BasketOfDeplorables
65,"TRUNEWS 10/27/16 Rob Kirby | Unspoken Secrets October 27, 2016 Will the media ever publicize the DARKEST secrets already released by Julian Assange? Today on TRUNEWS, Rick Wiles details the latest...",#x27
95,"On Thursday, the U.K. voted to leave the European Union, and, of course, conservatives here in the U.S. rejoiced because they see it as a move for freedom and sovereignty, and against big governme...",#BrexitVote #Wrongcountry #Scotsvotedtostay


**Clean leakage issue**

See notebook 2_LeakageTest for further info

In [85]:
# clean the references to (Reuters) from the text column
def clean_reuters_leakage(text):

  # get rid of reference to (Reuters) in text
  # some articles start out with the is a correction not followed by the (Reuters) reference and we are matching the whole string and substituting out
  pattern_reuters_1 = r"^[“”‘’A-Za-z.,&$()/\-:;0-9  ]*(\(Reuters\) - |\(Reuters\)\) - |\(Reuters\)  —  )"
  # we are matching first occurence so we are using match and not findall
  match_str_1 = re.match(pattern_reuters_1, text)
  if match_str_1:
    text = re.sub(pattern_reuters_1, '', text)
    #print(match_str_1.group())

  return text

In [86]:
# clean the (Reuters) reference out
data['text_clean'] = data['text_clean'].apply(clean_reuters_leakage)

***
##Sentence count and mean sentence length.
***

I want to drop the blank text_clean rows and make sure I have a decent mean sentence length for the langdetect package to work.

In [87]:
# using NLTK sentence tokenizer to count the number of sentences in each text passage in the 'text_clean' column
data.loc[:,'text_sent_count'] = data.loc[:,'text_clean'].map(lambda txt: len(sent_tokenize(txt)))

In [88]:
# check and see if there are any less than 1
# in the result dataframe the text column is just links with the text_clean column blank
data[data['text_sent_count'] < 1].head()

,title,text,label,title_clean,text_clean,email,links,link_count,mentions,mentions_count,hashtags,hashtag_count,text_sent_count
291,https://100percentfedup.com/12-yr-old-black-conservative-whose-video-to-obama-went-viral-do-you-really-love-america-receives-death-threats-from-left/,https://100percentfedup.com/12-yr-old-black-conservative-whose-video-to-obama-went-viral-do-you-really-love-america-receives-death-threats-from-left/,1,https://100percentfedup.com/12-yr-old-black-conservative-whose-video-to-obama-went-viral-do-you-really-love-america-receives-death-threats-from-left/,,,[https://100percentfedup.com/12-yr-old-black-conservative-whose-video-to-obama-went-viral-do-you-really-love-america-receives-death-threats-from-left/],1,,0,,0,0
660,TUCKER CARLSON Outs Human Rights Exec. Director As A Partisan Hack: “Obviously partisan motives diminish your mission” [Video],https://www.youtube.com/watch?v=VkRCtn0nEvU,1,TUCKER CARLSON Outs Human Rights Exec. Director As A Partisan Hack: “Obviously partisan motives diminish your mission”,,,[https://www.youtube.com/watch?v=VkRCtn0nEvU],1,,0,,0,0
2575,BREAKING: DEMOCRAT Makes Shocking Statement Regarding DNC Pick Keith Ellison [Video],https://www.youtube.com/watch?v=IioEIUmawRo,1,BREAKING: DEMOCRAT Makes Shocking Statement Regarding DNC Pick Keith Ellison,,,[https://www.youtube.com/watch?v=IioEIUmawRo],1,,0,,0,0
3904,https://fedup.wpengine.com/wp-content/uploads/2015/04/hillarystreetart.jpg,https://fedup.wpengine.com/wp-content/uploads/2015/04/hillarystreetart.jpg,1,https://fedup.wpengine.com/wp-content/uploads/2015/04/hillarystreetart.jpg,,,[https://fedup.wpengine.com/wp-content/uploads/2015/04/hillarystreetart.jpg],1,,0,,0,0
4315,Judge Jeanine Gets to The Bottom of “Towergate” [Video],https://www.youtube.com/watch?v=n9tfNMQpYWU,1,Judge Jeanine Gets to The Bottom of “Towergate”,,,[https://www.youtube.com/watch?v=n9tfNMQpYWU],1,,0,,0,0


In [89]:
# we will drop the rows less than 1
data.drop(data.loc[data['text_sent_count'] < 1].index, inplace=True)

In [90]:
# get mean sentence length
data.loc[:,'mean_sent_length'] = data.loc[:,'text_clean'].map(lambda txt: np.mean([len(sent) for sent in sent_tokenize(txt)]))

In [91]:
# look at rows with mean sentance length less than 20
# we will apply the language check and that needs a good amount of charaters
df = data.loc[data['mean_sent_length'] < 20,['text_clean','label']]
print(df)

                                                         text_clean  label
1858                                                 Read more: TMZ      1
2641                                                        Via: GP      1
4552                                      NO, NO. YOU RE FINISHED        1
5587                                                         Enjoy       1
6379   Notify the CDC. It's spreading.      BCP () October 14, 2016      1
6633       Wow! This is so epic and truthful! Bravo Bill O Reilly!       1
7307                                               : Gateway Pundit      1
8052             Unreal! Is he that out of it? It would appear so        1
8572                             Gary North has the video . 12:56        1
15568                                             Nice Admin Lady        1
18306                                                      Via: NPR      1
19266                                                Lefty losers        1
19863                    

In [92]:
# drop the low mean length articles
data.drop(data.loc[data['mean_sent_length'] < 20].index, inplace=True)

***
##Drop other languages
***

I want to keep only english rows

In [93]:
# runs for 16min
# this method will detect other languages using langdetect package
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'could not detect language'
# we are running the detect_language methon on the 'text_clean' column
data.loc[:,'lang'] = data.loc[:,'text_clean'].apply(detect_language)

In [94]:
# the count of english slightly changes every time I run the above method but the count should be around #61608
print(data.loc[:,'lang'].value_counts())

lang
en       61605
ru         156
es         141
de         100
fr          32
ar          19
pt           7
tr           7
it           4
hr           4
no           3
nl           3
el           2
pl           2
vi           1
sw           1
zh-cn        1
Name: count, dtype: int64


In [95]:
# show non english rows
df = data.loc[~data['lang'].eq('en')]
df.head()
#df.to_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning - Supervised Learning/project/ProjectData/lang.csv', index=False)

,title,text,label,title_clean,text_clean,email,links,link_count,mentions,mentions_count,hashtags,hashtag_count,text_sent_count,mean_sent_length,lang
172,"Les BRICS doivent faire face à la guerre financière des États-Unis, par Ariel Noyola Rodríguez","Les BRICS doivent faire face à la guerre financière des États-Unis par Ariel Noyola Rodríguez Pour faire face à la guerre financière provoquée par les États-Unis, il est urgent que les BRICS renfo...",1,"Les BRICS doivent faire face à la guerre financière des États-Unis, par Ariel Noyola Rodríguez","Les BRICS doivent faire face à la guerre financière des États-Unis par Ariel Noyola Rodríguez Pour faire face à la guerre financière provoquée par les États-Unis, il est urgent que les BRICS renfo...",,,0,,0,,0,30,210.766667,fr
383,Эксперт: Хэллоуин запретить невозможно,"0 комментариев 1 поделились Фото: AP \n""По мнению РПЦ, моему мнению, а также мнению ряда общественных организаций, празднование Хэллоуина в России нарушает действующее российское законодательство....",1,Эксперт: Хэллоуин запретить невозможно,"0 комментариев 1 поделились Фото: AP ""По мнению РПЦ, моему мнению, а также мнению ряда общественных организаций, празднование Хэллоуина в России нарушает действующее российское законодательство. ...",,,0,,0,,0,23,122.869565,ru
530,Новый уровень патриотизма,"Политика \nДмитрий Анатольевич Медведев знаком российскому обществу не как целомудренный политик, а как человек, ставящий превыше всего здоровье и хорошее настроение. Но на днях людям открылась др...",1,Новый уровень патриотизма,"Политика Дмитрий Анатольевич Медведев знаком российскому обществу не как целомудренный политик, а как человек, ставящий превыше всего здоровье и хорошее настроение. Но на днях людям открылась дру...",,,0,,0,,0,24,118.208333,ru
619,"Cómo decir «No» a las armas nucleares, por Manlio Dinucci",«El arte de la guerra»\nCómo decir «No» a las armas nucleares por Manlio Dinucci Estados Unidos acaba de imponer a sus aliados en la Comisión de Desarme y Seguridad de la Asamblea General de la ON...,1,"Cómo decir «No» a las armas nucleares, por Manlio Dinucci",«El arte de la guerra» Cómo decir «No» a las armas nucleares por Manlio Dinucci Estados Unidos acaba de imponer a sus aliados en la Comisión de Desarme y Seguridad de la Asamblea General de la ONU...,,,0,,0,,0,19,252.684211,es
643,Un conflit militaire entre la Russie et les Etats-Unis est-il possible ?,"Opinion Un Su-25 à la base aérienne russe de Hmeimim, Syrie. Crédit : AP \nLa crise syrienne est bien plus dangereuse que les crises de la guerre froide, y compris que la célèbre crise des missile...",1,Un conflit militaire entre la Russie et les Etats-Unis est-il possible ?,"Opinion Un Su-25 à la base aérienne russe de Hmeimim, Syrie. Crédit : AP La crise syrienne est bien plus dangereuse que les crises de la guerre froide, y compris que la célèbre crise des missiles...",,,0,,0,,0,61,132.557377,fr


In [96]:
# keep english rows
data = data.loc[data['lang'] == 'en']

# drop lang column
data = data.drop(columns=['lang'])

***
##Tokenize title and text
***

**This is where we finally tokenize the clean title and text**

In [97]:
# add additional stop words that aren't in the nltk library list
add_stop_words = {'also'}
print(add_stop_words)

# load the nltk library stop word list
stop_words = set(stopwords.words('english'))

# combine the 2 lists - don't really need this now because I did not add a lot of additional stop words but leaving in as place holder
stop_words = stop_words.union(add_stop_words)

# display stop words
print(stop_words)
print('said' in stop_words)

{'also'}
{'such', 'my', 'yours', 'll', 'm', "couldn't", "mightn't", 'under', "aren't", 'yourselves', 'each', 'couldn', 'these', 'ours', 'who', 'we', "weren't", "needn't", 'ma', 'here', 's', 'further', "she's", 'again', 'hers', 'an', 'against', 'theirs', 'other', 'he', 'off', 'not', "don't", 'all', 'been', 'hadn', "mustn't", "wouldn't", 'when', 'after', 'through', 'do', 'if', 'as', "it's", 'aren', 'into', 'those', "haven't", 'needn', 'about', 'below', "wasn't", 'for', 'didn', 'the', 've', "didn't", 'too', 'are', "hadn't", 'wouldn', 'own', "you'll", 'this', 'by', "won't", 'that', 'will', 'or', 'its', 'y', 'any', 'i', 'both', 'himself', 'no', 'does', 'won', 'him', 'were', 'having', 'd', 'there', 'before', 'you', 'in', "shouldn't", 'it', 'our', 'while', 'herself', 'once', 'up', 'a', 'can', 'shan', 'until', 'she', "you'd", 'so', 'myself', 'wasn', 'weren', 'had', 't', 'shouldn', 'hasn', 'then', 'your', 'but', 'don', 'be', 'at', 'between', "hasn't", 'over', 'am', 'haven', 'of', 'nor', 'his', 

In [98]:
# clean digit only strigs and get rid of other characters that are not words and tokenize string
def clean_tokens(text):
  # text to lower
  text_clean = text.lower()

  # get rid of digit only text strings
  text_clean = re.sub(r"\d+", '', text_clean)

  # set non-words to ''
  text_clean = re.sub("\W", ' ', text_clean)

  # tokenize text
  tokens = word_tokenize(text_clean)

  # check to make sure length of token is greater than 1 and combine into a new text string
  # I want to combine into text string to be able to save to csv file and load later with minimal processing
  text_list = [word for word in tokens if word not in stop_words and len(word)>1]
  token_to_text = ' '.join(text_list)

  # return text string
  return token_to_text

In [99]:
# test clean_tokens method
text1 = data.loc[0,'text_clean']
print(text1)
print(clean_tokens(text1))
print('--------------')
title1 = data.loc[0,'title_clean']
print(title1)
print(clean_tokens(title1))

Written by Philip Giraldi Tuesday November 8, 2016 The American people don’t know very much about war even if Washington has been fighting on multiple fronts since 9/11. The continental United States has not experienced the presence a hostile military force for more than 100 years and war for the current generation of Americans consists largely of the insights provided by video games and movies. The Pentagon’s invention of embedded journalists, which limits any independent media insight into what is going on overseas, has contributed to the rendering of war as some kind of abstraction. Gone forever is anything like the press coverage of Vietnam, with nightly news and other media presentations showing prisoners being executed and young girls screaming while racing down the street in flames. Given all of that, it is perhaps no surprise that both Hillary Clinton and Donald Trump, neither of whom has served in uniform, should regard violence inflicted on people overseas with a considerable

In [100]:
# tokenize the cleaned title and text columns
data.loc[:,'title_tokens_to_text'] = data.loc[:,'title_clean'].apply(clean_tokens)
data.loc[:,'text_tokens_to_text'] = data.loc[:,'text_clean'].apply(clean_tokens)

***
##Check recognized words and non-ascii characters
***

**I am checking to see if there are unrecognized words. While looking for unrecognized words I found more foreign words. Most of the foreign words are non-ascii so I removed those to create a recogized_words_to_text column. This is probably over kill, but I wanted to look at the percentage of non_recognized words to total_words.**

In [101]:
# load saved csv file of other recognized words that are not included in nltk or wordnet word or lemmas collections
# the saved addl word list was taken from a prior run resulting in a non recognized word and then running through MS Word to see if recognized by spell check and if it was not considered misspelled then I added to addl word list.
file = open('/content/drive/MyDrive/Colab Notebooks/Machine Learning - Supervised Learning/project/ProjectData/addl_words.txt', 'r', encoding='utf-8-sig')

# parse csv file with ', '
addl_words = file.read().split(', ')

# strip the extra ' characters
addl_words = [word.strip('\'') for word in addl_words]
add_words = set(addl_words)
file.close()

print(add_words)
print(len(add_words)) # the count should be 49483

{'zwigoff', 'bejesus', 'stanislavski', 'magliozzi', 'sooper', 'garrido', 'nacchio', 'meridith', 'badghis', 'barstow', 'dodged', 'shoney', 'eggo', 'valium', 'leboeuf', 'christmas', 'gunderson', 'danang', 'lanier', 'pantaleo', 'mykola', 'liberating', 'muoio', 'nicolae', 'columbian', 'graney', 'denisova', 'gunworks', 'cecil', 'plasticky', 'carinthia', 'katainen', 'coi', 'wikler', 'rason', 'martelly', 'thrummed', 'springform', 'jivin', 'cammarata', 'zullo', 'faustin', 'batek', 'neom', 'maksim', 'methuen', 'kurten', 'taobao', 'cg', 'thorsten', 'shiro', 'lopatin', 'folha', 'alavi', 'englebright', 'lincoln', 'salva', 'croda', 'gamesa', 'lauding', 'corte', 'karam', 'rolodexes', 'susin', 'headey', 'icarus', 'aneth', 'bowden', 'globalizers', 'deandra', 'nohl', 'newseum', 'riegle', 'guinan', 'kollman', 'masisi', 'chesler', 'radionuclide', 'meerbusch', 'begrudges', 'antonello', 'pomeranian', 'dufty', 'toal', 'deserves', 'obermayer', 'corbyn', 'belmond', 'baathist', 'essanay', 'bugti', 'casciaro', 

In [102]:
# get NLTK word list
nltk.download('words')
word_list = set(words.words())

# get lemmas from wordnet
# get all synsets from wordnet.all_synsets() then get the lemma names from each synset
# this will help us expand the words list with different words and we just want to add the lemmas
# when we check the words we will take down to lemma form if there is not a direct match right away
wordnet_lemmas = set(lemma.name() for synset in wordnet.all_synsets() for lemma in synset.lemmas())
# combine lists
word_list_master = word_list.union(wordnet_lemmas)
word_list_master = word_list_master.union(add_words)
print(len(word_list_master)) # the count should be 376754

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


376754


In [103]:
# initiate a wordnet lemmatizer
wnl = WordNetLemmatizer()

# method to check if word is a recognized word
def check_words(tokens_to_text):
  # this is a part of speach dictionary to match coding between NLTK
  # maps part-of-speech tags (like 'NN' for noun, 'VB' for verb) to simplified tags ('n', 'v', 'a', 'r') used by WordNet
  pos_ref = {'NN': 'n', 'NNP': 'n', 'NNPS': 'n', 'NNS': 'n', 'JJ': 'a', 'JJR': 'a', 'JJS': 'a', 'RB': 'r', 'RBR': 'r', 'RBS': 'r', 'RP': 'r', 'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v'}
  token_lst = tokens_to_text.split()
  # list of tokens with pos tags using nltk.pos_tag method
  lst_pos_tags = nltk.pos_tag(token_lst)

  # set of non words or mispelled words
  not_in_words = set()
  # set of non ascii charater strings because I could not get rid of all foreign words about and want to see where they are at
  not_ascii = set()
  # add recognized words to list
  recog_words = []

  # loop through tokens
  for token in lst_pos_tags:
    # if word in master word list add to recog_words
    if token[0] in word_list_master:
      recog_words.append(token[0])
      pass
    # if word is not recognized right away check is lemma and see if it is recognized
    elif wnl.lemmatize(token[0]) in word_list_master:
      recog_words.append(token[0])
      pass
    # otherwise look at token and pos tag
    else:
      word = token[0]
      # get wordnet pos from NLTK pos tag
      pos = pos_ref.get(token[1])
      # is pos tag is one from the dictionary above then check lemma using lematize with the pos information
      if pos is not None:
        lemma = wnl.lemmatize(word, pos)
        if lemma in word_list_master:
          recog_words.append(token[0])
          pass
        # if the token is still not recognized then check to see if it is ascii
        else:
          if token[0].isascii():
            not_in_words.add(token[0])
          else:
            not_ascii.add(token[0])
            #print(token[0]) #printing all non-ascii tokens
      # for tokens not labeled with pos that is in above pos_ref dictionary then check to see if the token is isascii
      else:
        if token[0].isascii():
          not_in_words.add(token[0])
        else:
          not_ascii.add(token[0])
          #print(token[0])
  recog_words_to_text = ' '.join(recog_words)
  # return recog_words_to_text and the not_in_words set and not_ascii set as columns
  return pd.Series({'recog_words_to_text': recog_words_to_text, 'text_not_words': not_in_words, 'not_ascii': not_ascii})

In [104]:
# test check_words methon
text1 = data.loc[0,'text_tokens_to_text']
print(text1)

good, bad, not_ascii = check_words(text1)
print(good)
print(bad)
print(not_ascii)

written philip giraldi tuesday november american people know much war even washington fighting multiple fronts since continental united states experienced presence hostile military force years war current generation americans consists largely insights provided video games movies pentagon invention embedded journalists limits independent media insight going overseas contributed rendering war kind abstraction gone forever anything like press coverage vietnam nightly news media presentations showing prisoners executed young girls screaming racing street flames given perhaps surprise hillary clinton donald trump neither served uniform regard violence inflicted people overseas considerable level detachment hillary notorious assessment brutal killing libya moammar gaddafi saying came saw died share extent dominant new york washington policy consensus view dealing foreigners sometimes get bit bloody price someone power prepared pay one hillary top advisers former secretary state madeleine alb

In [105]:
# run check_words method and get new columns
data[['recog_words_to_text', 'text_not_words', 'not_ascii']] = data['text_tokens_to_text'].apply(lambda x: pd.Series(check_words(x)))
# run for approx 20min

In [106]:
# count the number of non_ascii tokens
data.loc[:,'non_ascii_count'] = data.loc[:,'not_ascii'].map(lambda lst: len(lst))

In [107]:
# see which label has the most rows with high _non_ascii_counts
# this is most likely because they have text that is a mix of english and foreign words
df_no_ascii_high = data[data['non_ascii_count'] > 5]
print(df_no_ascii_high['label'].value_counts())

label
1    42
0    19
Name: count, dtype: int64


In [108]:
# calculate non_regog_word count and the percentage that is not recognized
data.loc[:,'non_recog_word_count'] = data.loc[:,'text_not_words'].map(lambda lst: len(lst))
data.loc[:,'total_word_count'] = data.loc[:,'text_tokens_to_text'].map(lambda lst: len(lst))
data['non_word_percent'] = (data['non_recog_word_count'] + data['non_ascii_count']) / data['total_word_count'] * 100

**I did expect the real articles to have a lower non-word percentage**

In [109]:
# compare non_word_percent by label
print(data.loc[data['label'] == 0, 'non_word_percent'].mean())
print(data.loc[data['label'] == 1, 'non_word_percent'].mean())

0.13381513238181678
0.19975176914537837


***
##Remove non-ascii tokens out of text tokens
***

In [110]:
# method to clean out the non_ascii characters 'text_token_to_text' column
def clean_ascii(text_tokens, not_ascii):
  words = text_tokens.split()
  text_ascii = [word for word in words if word not in not_ascii]
  text_clean_ascii = ' '.join(text_ascii)
  return text_clean_ascii

In [111]:
# test above method
text_clean = data.loc[7666,'text_tokens_to_text']
not_ascii = data.loc[7666,'not_ascii']
print(clean_ascii(text_clean, not_ascii))

st century wire says week historic international iranian nuclear deal punctuated two way prisoner swap washington tehran end quite way everyone expected iranian side one citizens detained iran nosratollah khosravi roodsari stayed iran side iranians held prisons show flight geneva prisoner exchange least electing stay tehran side iran prisoners released making way germany via switzerland robinson daily mailnone iranians freed prisoner swap returned home could still united states reported seven former inmates released part deal islamic republic show get flight geneva switzerland exchange set take place sunday three iranians decided stay united states abc reported moving families however known four three americans detained iran washington post journalist jason rezaian former marine amir hekmati christian pastor saeed abedini left tehran around day met counterparts switzerland continue story mail onlineread iran news st century wire iran files


In [112]:
# create a new column of clean tokens that do not have non-ascii tokens
data.loc[:,'text_clean_ascii'] = data.loc[:,['text_tokens_to_text', 'not_ascii']].apply(lambda row: clean_ascii(row['text_tokens_to_text'], row['not_ascii']), axis=1)

***
##Title and Text Similarity with word2vec
***

**I thought it would be helpful to look at the similarity between title and text.**

In [113]:
# check to see rows with 'title_tokens_to_text' values blank
# I need to keep the rows
df = data.loc[data['title_tokens_to_text'].str.len()<1,['title_clean','text_clean','title_tokens_to_text','text_tokens_to_text','label']]
print(df)

                 title_clean  \
17774  It’s Not You, It’s Me   
32173                      :   
34463              What Now?   
50885         Won, Now What?   
55629              What If….   
57739            If It’s She   

                                                                                                                                                                                                    text_clean  \
17774  Kim Severson is filling in for Sam Sifton, who emails readers of Cooking five days a week to talk about food and suggest recipes. That email also appears here. To receive it in your inbox, registe...   
32173   We the People Against Tyranny: Seven Principles for Free Government By John W. Whitehead As I look at America today, I am not afraid to say that I am afraid.Former presidential advisor Bertr...   
34463  by Thomas Sowell  The good news is that we dodged a bullet in this election. The bad news is that we don’t know how many other bullets are

In [114]:
# we will drop the rows less than 1
data.drop(data.loc[data['title_tokens_to_text'].str.len() < 1].index, inplace=True)

In [115]:
# list of tokens from text and title
# this is to establish a vocabulary
tokens_collection_title = [token_lst.split() for token_lst in data.loc[:,'title_tokens_to_text']]
tokens_collection_text = [token_lst.split() for token_lst in data.loc[:,'text_tokens_to_text']]
tokens_collection = tokens_collection_title + tokens_collection_text
print(tokens_collection[0:10])

[['oh', 'lovely', 'war'], ['deported', 'italian', 'mobster', 'caught', 'sneaking', 'across', 'mexico', 'border'], ['china', 'lodges', 'protest', 'trump', 'call', 'taiwan', 'president'], ['us', 'may', 'soon', 'face', 'apocalyptic', 'seismic', 'event'], ['joy', 'behar', 'still', 'claims', 'clinton', 'wore', 'bizarre', 'mourning', 'item', 'hillary', 'lost'], ['poll', 'trump', 'pulls', 'ahead', 'florida'], ['canadian', 'groups', 'seek', 'overturn', 'quebec', 'ban', 'muslim', 'veil'], ['supreme', 'court', 'strikes', 'texas', 'abortion', 'law', 'anti', 'choicers', 'positively', 'fuming', 'tweets'], ['breaking', 'democrat', 'congressman', 'vocal', 'illegal', 'alien', 'advocate', 'trump', 'impeachment', 'leader', 'luis', 'gutiérrez', 'announce', 'run', 'election', 'behind', 'sudden', 'decision'], ['christian', 'family', 'kills', 'year', 'old', 'mom', 'attempted', 'exorcism']]


In [116]:
# make a trained word2vec model from the title and text tokens
# vector_size is the dimensionality of the word - the higher dimension can capture more complex relationships
# window determines how many words back and forward to look around the word
word2vec_model = Word2Vec(tokens_collection, vector_size=300, window=5, min_count=2, workers=4)

In [119]:
def title_text_similarity(title, text, model=word2vec_model):
  title_tokens = title.split()
  text_tokens = text.split()

  #the title_vec and text_vec need to be at same size as vector_size in the word2vec_model
  title_vec = np.zeros(300)
  text_vec = np.zeros(300)

  # loop through title tokens
  for token in title_tokens:
    # if token in word2vec model add to title_vec
    if token in model.wv:
      title_vec = np.add(title_vec, model.wv[token])

  # loop through text tokens
  for token in text_tokens:
    # if token in word2vec model add to title_vec
    if token in model.wv:
      text_vec = np.add(text_vec, model.wv[token])


  # if either title_vec or text_vec is a zero vector return 0
  if np.linalg.norm(title_vec) == 0 or np.linalg.norm(text_vec) == 0:
      return 0  # or any other default value you prefer
  else:
      # similarity calculation = (dot product of title_vec and text_vec) / (magnitude of title_vec * magnitude of text_vec)
      # cosine similarity is calculated by dividing the dot product of two vectors by the product of their magnitudes
      return round(np.dot(title_vec, text_vec) / (np.linalg.norm(title_vec) * np.linalg.norm(text_vec)), 4)

In [120]:
# test title_text_similarity method
print(data.loc[1,'title_tokens_to_text'])
print(data.loc[1,'text_tokens_to_text'])

print(title_text_similarity(data.loc[1,'title_tokens_to_text'], data.loc[1,'text_tokens_to_text']))

deported italian mobster caught sneaking across mexico border
previously deported italian mobster arrested trying illegally enter sneaking across porous border mexico man originally deported serving time federal prison connection drug trafficking violent assaults arrest took place near border city nogales arizona border patrol agents arrested salvatore marciante tried sneak country arrest agents able search marciante criminal history confirmed permanent legal resident living new york convictions lost legal status serving several years prison deported italy according federal court records obtained breitbart texas marciante citizen italy traveled mexico order enter country nogales arizona marciante deported september new york currently facing illegal charges federal courthouse tucson arizona marciante originally named indictment filed accusing members italian mafia distributing trafficking narcotics new york many documents listed court proceedings remain sealed court available court docu

In [121]:
# calculate title_text_similarity on all rows
data.loc[:,'title_text_similarity'] = data.loc[:,['title_tokens_to_text', 'text_tokens_to_text']].apply(lambda row: title_text_similarity(row['title_tokens_to_text'], row['text_tokens_to_text']), axis=1)

**I did expect the real articles to have a higher title to text similarity score**

In [122]:
# check how title_text_similarity compare based on label
print(data.loc[data['label']==0, 'title_text_similarity'].mean())
print(data.loc[data['label']==1, 'title_text_similarity'].mean())

0.6767633262075341
0.6426992032020161


***
##Save data
***

In [123]:
data.head()

,title,text,label,title_clean,text_clean,email,links,link_count,mentions,mentions_count,...,text_tokens_to_text,recog_words_to_text,text_not_words,not_ascii,non_ascii_count,non_recog_word_count,total_word_count,non_word_percent,text_clean_ascii,title_text_similarity
0,"Oh, What a Lovely War!","Written by Philip Giraldi Tuesday November 8, 2016 The American people don’t know very much about war even if Washington has been fighting on multiple fronts since 9/11. The continental United Sta...",1,"Oh, What a Lovely War!","Written by Philip Giraldi Tuesday November 8, 2016 The American people don’t know very much about war even if Washington has been fighting on multiple fronts since 9/11. The continental United Sta...",,,0,,0,...,written philip giraldi tuesday november american people know much war even washington fighting multiple fronts since continental united states experienced presence hostile military force years war...,written philip giraldi tuesday november american people know much war even washington fighting multiple fronts since continental united states experienced presence hostile military force years war...,"{ed, op, madeleine, stavridis}",{},0,4,8976,0.044563,written philip giraldi tuesday november american people know much war even washington fighting multiple fronts since continental united states experienced presence hostile military force years war...,0.2733
1,Deported Italian Mobster Caught Sneaking Across U.S.-Mexico Border,A previously deported Italian mobster has been arrested trying to illegally enter the U. S. by sneaking across the porous border with Mexico. The man had originally been deported after serving tim...,0,Deported Italian Mobster Caught Sneaking Across U.S.-Mexico Border,A previously deported Italian mobster has been arrested trying to illegally enter the U. S. by sneaking across the porous border with Mexico. The man had originally been deported after serving tim...,,,0,,0,...,previously deported italian mobster arrested trying illegally enter sneaking across porous border mexico man originally deported serving time federal prison connection drug trafficking violent ass...,previously deported italian mobster arrested trying illegally enter sneaking across porous border mexico man originally deported serving time federal prison connection drug trafficking violent ass...,{marciante},{},0,1,1665,0.060060,previously deported italian mobster arrested trying illegally enter sneaking across porous border mexico man originally deported serving time federal prison connection drug trafficking violent ass...,0.4540
2,China lodges protest after Trump call with Taiwan president,"BEIJING/WASHINGTON (Reuters) - China lodged a diplomatic protest on Saturday after U.S. President-elect Donald Trump spoke by phone with President Tsai Ing-wen of Taiwan, but blamed the self-ruled...",0,China lodges protest after Trump call with Taiwan president,"China lodged a diplomatic protest on Saturday after U.S. President-elect Donald Trump spoke by phone with President Tsai Ing-wen of Taiwan, but blamed the self-ruled island Beijing claims as its o...",,,0,,0,...,china lodged diplomatic protest saturday president elect donald trump spoke phone president tsai ing wen taiwan blamed self ruled island beijing claims petty move minute telephone call taiwan lead...,china lodged diplomatic protest saturday president elect donald trump spoke phone president tsai ing wen taiwan blamed self ruled island beijing claims petty move minute telephone call taiwan lead...,{cnn},{},0,1,4195,0.023838,china lodged diplomatic protest saturday president elect donald trump spoke phone president tsai ing wen taiwan blamed self ruled island beijing claims petty move minute telephone call taiwan lead...,0.8456
3,The US May Soon Face an Apocalyptic Seismic Event,"Today, an ever increasing number of earthquakes in the United States may soon bring the country to ruin, as geologists, journalists and politicians say. \nVia UsualRoutine \n\nThe Universi

In [124]:
# save to csv
data.to_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning - Supervised Learning/project/ProjectData/DataClean.csv', index=False)

In [125]:
# remaining label counts
print(data['label'].value_counts())
# label counts should be the following, but can be slightly different because of the variability in how langdetect works
#0    34615
#1    26983

label
0    34616
1    26983
Name: count, dtype: int64
